# HLA

- allele -> sequence

In [4]:
import re
from Bio import SeqIO

fasta_name = "/data/lujd/TCRdata/raw/imgt/hla_prot.fasta"
records = SeqIO.parse(fasta_name, "fasta")

name2seq_dict = {}
hlaI_name_pattern = r"^[A-G]\*"
name_preffix = "HLA-"
short_position = [7,9,24,45,59,62,63,66,67,69,70,73,74,76,77,80,81,84,95,97,99,
                114,116,118,143,147,150,152,156,158,159,163,167,171]        # netmhcpan
short_position = [pos-1 for pos in short_position]

for record in records:
    info = {}
    name = record.description.split(' ')[1]
    if bool(re.match(hlaI_name_pattern, name)):
        # full 
        full_len = int(record.description.split(' ')[2])
        full_seq = str(record.seq)
        info['full sequence'] = full_seq
        info['full sequence length'] = full_len
        assert len(full_seq) == full_len

        # clip (require seq length > 180)
        if (full_len == 180) or (full_len == 181):
            info['clip sequence'] = full_seq
            info['clip sequence length'] = full_len
        elif full_len > 181:
            SH_position = full_seq.find("SH")
            if SH_position == 0:
                clip_seq = full_seq[SH_position:181]                 # SH...
                info['clip sequence'] = clip_seq
                info['clip sequence length'] = len(clip_seq)
            elif SH_position > 0:
                clip_seq = full_seq[SH_position-1:SH_position+181]   # G/CSH...
                if len(clip_seq) >= 180:
                    info['clip sequence'] = clip_seq
                    info['clip sequence length'] = len(clip_seq)
                else:
                    info['clip sequence'] = '(UNK)'
                    info['clip sequence length'] = 0
        else:
            info['clip sequence'] = '(UNK)'
            info['clip sequence length'] = 0

        # short
        if info['clip sequence length'] > 0:
            if info['clip sequence length'] < 182:
                if info['clip sequence'].startswith('SH'):
                    short_seq = ""
                    for ind in short_position:
                        short_seq += info['clip sequence'][ind-1]
                    info['short sequence'] = short_seq
                    info['short sequence length'] = len(short_seq)
                else:
                    info['short sequence'] = '(UNK)'
                    info['short sequence length'] = 0
            else:
                short_seq = ""
                for ind in short_position:
                    short_seq += info['clip sequence'][ind]
                info['short sequence'] = short_seq
                info['short sequence length'] = len(short_seq)
        else:
            info['short sequence'] = '(UNK)'
            info['short sequence length'] = 0
        
        # save
        print(name_preffix+name, info['clip sequence length'], 
                    info['clip sequence'][:5], info['clip sequence'][-5:],
                    info['short sequence length'], info['short sequence'])
        name2seq_dict[name_preffix+name] = info

HLA-A*01:01:01:01 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:02N 0 (UNK) (UNK) 0 (UNK)
HLA-A*01:01:01:03 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:04 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:05 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:06 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:07 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:08 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:09 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:10 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:11 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:12 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:13 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:14 182 GSHSM TLQRT 34 YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
HLA-A*01:01:01:15 182 G

In [5]:
import os
import json

savepath = "/data/lujd/TCRdata/collect"
savename = "hla_allele2seq.json"

with open(os.path.join(savepath, savename), "w") as f:
    json.dump(name2seq_dict, f, indent=4)

print(F"File {os.path.join(savepath, savename)} has been successfully saved.")

File /data/lujd/TCRdata/collect/hla_allele2seq.json has been successfully saved.


---

# TCR

- gene name -> sequence

In [24]:
import re
import os
from Bio import SeqIO

fasta_path = "/data/lujd/TCRdata/raw/imgt/"
fasta_names = ['trav_prot_human.fasta','traj_prot_human.fasta','trbv_prot_human.fasta','trbd_prot_human.fasta','trbj_prot_human.fasta']

name2seq_dict = {}
for fasta_name in fasta_names:
    print(fasta_name)
    records = SeqIO.parse(os.path.join(fasta_path, fasta_name), "fasta")
    for record in records:
        info = {}
        gene_name = record.description.split('|')[1]
        species = record.description.split('|')[2]
        full_len = int(record.description.split('|')[11].split(' ')[0])     # 99 AA
        full_seq = str(record.seq)
        assert full_len == len(full_seq)
        print(gene_name, full_len, full_seq)

        info['species'] = species
        info['amino acid sequence'] = full_seq
        info['sequence length(aa)'] = full_len

        if gene_name in name2seq_dict.keys():
            print(f"There's already an identical {gene_name}.")
        else:
            name2seq_dict[gene_name] = info
    print(len(name2seq_dict))

trav_prot_human.fasta
TRAV1-1*01 91 GQSLEQPSEVTAVEGAIVQINCTYQTSGFYGLSWYQQHDGGAPTFLSYNALDGLEETGRFSSFLSRSDSYGYLLLQELQMKDSASYFCAVR
TRAV1-1*02 89 GQSLEQPSEVTAVEGAIVQINCTYQTSGFYGLSWYQQHDGGAPTFLSYNGLDGLEETGRFSSFLSRSDSYGYLLLQELQMKDSASYFCA
TRAV1-2*01 91 GQNIDQPTEMTATEGAIVQINCTYQTSGFNGLFWYQQHAGEAPTFLSYNVLDGLEEKGRFSSFLSRSKGYSYLLLKELQMKDSASYLCAVR
TRAV1-2*02 58 GQNIDQPTEMTATEGAIVQINCTYQTSGFNGLFWYQQHAGEAPTFLSYNVLDGLEEKG
TRAV1-2*03 91 GQNIDQPTEMTATEGAIVQINCTYQTSGFNGLFWYQQHAGEAPTFLSYNVLDGLEEKGRFSSFLSRSKGYSYLLLKELQMKDSASYLCAVR
TRAV10*01 93 KNQVEQSPQSLIILEGKNCTLQCNYTVSPFSNLRWYKQDTGRGPVSLTIMTFSENTKSNGRYTATLDADTKQSSLHITASQLSDSASYICVVS
TRAV10*02 93 KNQVEQSPQSLIILEGKNCTLQCNYTVSPFSNLRWYKQDTGRGPVSLTIMTFSENTKSNGRYTATLDADTKQSSLHITASQLSDSASYICVVS
TRAV11*01 92 LHTLEQSPSFLNIQEGMHAVLNCTYQERTLFNFHWFRQDPGRRLVSLTLIQSSQKEQGDKYFKELLGKEKFYSVWNIAASHLGDSATYFCAL
TRAV12-1*01 91 RKEVEQDPGPFNVPEGATVAFNCTYSNSASQSFFWYRQDCRKEPKLLMSVYSSGNEDGRFTAQLNRASQYISLLIRDSKLSDSATYLCVVN
TRAV12-1*02 91 RKEVEQDPGPFNVPEGATVAFNCTYSNSASQSFFWYRQDCR

In [25]:
import os
import json

savepath = "/data/lujd/TCRdata/collect"
savename = "tcr_gene2seq.json"

with open(os.path.join(savepath, savename), "w") as f:
    json.dump(name2seq_dict, f, indent=4)

print(F"File {os.path.join(savepath, savename)} has been successfully saved.")

File /data/lujd/TCRdata/collect/tcr_gene2seq.json has been successfully saved.
